**Research Questions**


*   Can we determine whether attending a charter school from 3rd to 5th grade improves the 5th-grade reading performance of economically disadvantaged students compared to attending a public school, by using a logistic regression model? The model will be trained on data from economically disadvantaged students who attended public schools continuously from 3rd to 5th grade in Texas. The analysis will use the Texas Standardized Exam as the performance metric. 

**Features as Predictors**
* List provided at bottom of code

**Predicted Variables**
* 2019 Reading Meets

In [48]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.metrics import  auc


from statsmodels.stats.contingency_tables import mcnemar
from scipy.stats import chi2_contingency
import xgboost as xgb

In [49]:
# import data
df = pd.read_csv('STAAR_Student_Level _2017_2023_Full.csv')

C:\Users\Cesar\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (108) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [50]:
# drop all students with no grade data for the year 2017
df.dropna(subset=['2017 Grade'], inplace=True)

In [51]:
# First round of drops
## we will only be looking data related to reading for the years 2017 and 2019
df.drop(['2018 Math Did Not Meet',
 '2019 Math Approaches',
 '2019 Math Meets',
 '2019 Math Masters',
 '2019 Math Overall Score','2021 Year',
 '2021 Month',
 '2021 Grade',
 '2021 District ID',
 '2021 District Name',
 '2021 Campus Name',
 '2021 Campus ID',
 '2021 Sex',
 '2021 Ethnicity',
 '2021 EcoDis',
 '2021 Reading Did Not Meet',
 '2021 Reading Approaches',
 '2021 Reading Meets',
 '2021 Reading Masters',
 '2021 Reading Overall Score',
 '2021 Math Did Not Meet',
 '2021 Math Approaches',
 '2021 Math Meets',
 '2021 Math Masters',
 '2021 Math Overall Score',
 '2021 EOC',
 '2022 Year',
 '2022 Month',
 '2022 Grade',
 '2022 District ID',
 '2022 District Name',
 '2022 Campus Name',
 '2022 Campus ID',
 '2022 Sex',
 '2022 Ethnicity',
 '2022 EcoDis',
 '2022 Reading Did Not Meet',
 '2022 Reading Approaches',
 '2022 Reading Meets',
 '2022 Reading Masters',
 '2022 Reading Overall Score',
 '2022 Math Did Not Meet',
 '2022 Math Approaches',
 '2022 Math Meets',
 '2022 Math Masters',
 '2022 Math Overall Score',
 '2022 EOC',
 '2023 Year',
 '2023 Month',
 '2023 Grade',
 '2023 District ID',
 '2023 District Name',
 '2023 Campus ID',
 '2023 Campus Name',
 '2023 Sex',
 '2023 Ethnicity',
 '2023 EcoDis',
 '2023 Reading Did Not Meet',
 '2023 Reading Approaches',
 '2023 Reading Meets',
 '2023 Reading Masters',
 '2023 Math Did Not Meet',
 '2023 Math Approaches',
 '2023 Math Meets',
 '2023 Math Masters',
 '2023 Reading Overall Score',
 '2023 Math Overall Score',
 '2023 EOC'], axis=1, inplace=True)

In [52]:
# Check for duplicates 
df.duplicated().sum()

# Drop duplicates and only keep the first 
df.drop_duplicates(keep='first', inplace=True)

In [53]:
# Columns relevant to "Reading Overall Score" across 2017 and 2019
reading_score_columns = []
years = [2017, 2019]

for year in years:
    reading_score_col = f'{year} Reading Overall Score'
    if reading_score_col in df.columns:
        reading_score_columns.append(reading_score_col)
        
# Drop rows where any of the identified reading score columns have missing values
df = df.dropna(subset=reading_score_columns)

In [54]:
# Define a function to categorize district types based on the 4th character in the District ID
def categorize_district(district_id):
    if pd.isna(district_id):
        return None
    elif district_id[4] == '8':
        return 'Charter'
    elif district_id[4] == '9':
        return 'Public'
    else:
        return 'Unknown'

In [55]:
# Apply this function to each district ID column from each year and create a new flag column for each
for year in range(2017, 2020):
    district_id_column = f"{year} District ID"
    if district_id_column in df.columns:
        df[f"{year} School Type"] =df[district_id_column].apply(categorize_district)

In [56]:
# Define a function to determine if a student stayed in the same type of school (Charter or Public) across all years
def consistent_school_type(student_record):
    types = [student_record[f"{year} School Type"] for year in range(2017, 2020) if f"{year} School Type" in student_record]
    # Filter out 'None' which represents years not in school
    types = [t for t in types if t is not None]
    if len(set(types)) == 1:  # All entries are the same and not empty
        return types[0]  # Return the consistent type
    return 'Mixed'  # Mixed types or no data

In [57]:
# Apply this function to each row in the DataFrame to create a new column for overall school type consistency
df['Consistent School Type'] = df.apply(consistent_school_type, axis=1)

In [58]:
#drop students that have a Mixed or Unknown Consistent School Type
## We only need students who attended one form of school (Charter or Public)
values = ['Mixed', 'Unknown']

df = df[df['Consistent School Type'].isin(values) == False]

In [59]:
# Only need students that atteded the 3rd grade in 2017 and the 5th grade in 2019

df = df[(df['2017 Grade'] == 3) & (df['2019 Grade'] == 5)]

In [60]:
# Drop Math data for the 3rd grade and all but Reading Results for the 5th grade
df.drop(['2017 Campus Name',
 '2017 Math Did Not Meet',
 '2017 Math Approaches',
 '2017 Math Meets',
 '2017 Math Masters',
 '2017 Math Overall Score',
 '2018 Year',
 '2018 Month',
 '2018 Grade',
 '2018 District ID',
 '2018 District Name',
 '2018 Campus Name',
 '2018 Campus ID',
 '2018 Sex',
 '2018 Ethnicity',
 '2018 EcoDis',
 '2018 Reading Did Not Meet',
 '2018 Reading Approaches',
 '2018 Reading Meets',
 '2018 Reading Masters',
 '2018 Reading Overall Score',
 '2018 Math Approaches',
 '2018 Math Meets',
 '2018 Math Masters',
 '2018 Math Overall Score',
 '2018 EOC',
 '2019 Year',
 '2019 Month',
 '2019 Grade',
 '2019 District ID',
 '2019 District Name',
 '2019 Campus Name',
 '2019 Campus ID',
 '2019 Sex',
 '2019 Ethnicity',
 '2019 EcoDis',
 '2019 Reading Did Not Meet',
 '2019 Reading Approaches',
 '2019 Reading Masters',
 '2019 Reading Overall Score',
 '2019 Math Did Not Meet',
 '2019 EOC',
 '2018 School Type'], axis=1, inplace=True)

In [61]:
df['2017 Grade'] = df['2017 Grade'].astype(int).astype(str)
df['2017 Year'] = df['2017 Year'].astype(int).astype(str)

In [62]:
# Import Campus Rating Data
campus_rating = pd.read_excel('Campus_Rating.xlsx')

In [63]:
# Merge Campus rating data with student data
df_rating = df.merge(campus_rating, left_on="2017 Campus ID", right_on='Campus Number')

In [64]:
# Select Categorical Columns for encoding
categorical_columns= ['2017 Sex','2017 Ethnicity','2017 EcoDis','2017 Reading Overall Score','School Type',
 'Campus 2017 Rating']

In [65]:
# One-hot encoding the categorical columns
encoder = OneHotEncoder(sparse=False)
encoded_categorical = encoder.fit_transform(df_rating[categorical_columns])

C:\Users\Cesar\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [66]:
# Creating a DataFrame from the encoded variables
encoded_categorical_df = pd.DataFrame(
    encoded_categorical,
    columns=encoder.get_feature_names_out(categorical_columns)
)

In [67]:
df_reset = df_rating.reset_index()

In [68]:
# Concat Student data with endcoded data
df_encode = pd.concat([df_reset, encoded_categorical_df], axis=1)

In [69]:
# Select Encoded and numerical colomns 
df = df_encode[['2017 Campus ID','Student ID','2019 Reading Meets', 'Consistent School Type', '2017 Sex_F',
       '2017 Sex_M', '2017 Ethnicity_Black', '2017 Ethnicity_Hispanic',
       '2017 Ethnicity_Other Race', '2017 Ethnicity_White',
       '2017 EcoDis_EcoDis', '2017 EcoDis_Non-EcoDis',
       '2017 Reading Overall Score_Approaches',
       '2017 Reading Overall Score_Did Not Meet',
       '2017 Reading Overall Score_Masters',
       '2017 Reading Overall Score_Meets', 'School Type_B', 'School Type_E',
       'Campus 2017 Rating_A', 'Campus 2017 Rating_I', 'Campus 2017 Rating_M',
       'Campus 2017 Rating_T']]

In [70]:
# Import campus features data
campus = pd.read_excel('CAMPPROF_Merge.xlsx')

In [71]:
# Convert data to numeric and non numeric data to NaNs
cols = campus.columns.drop('Campus Number')

campus[cols] = campus[cols].apply(pd.to_numeric, errors='coerce')

In [72]:
#Find columns with large number of NaNs
campus_NaCounts = campus.isna().sum()
campus_NaCounts.sort_values(ascending=False)

Campus 2017 Class Size: Mixed Elem. - Avg Size                                           7948
Campus 2017 Class Size: Grade 6 - Avg Size                                               6216
Campus 2017 Class Size: Sec For Lng - Avg Size                                           5886
Campus 2017 Class Size: Sec Soc Stud- Avg Size                                           5214
Campus 2017 Class Size: Sec Sci - Avg Size                                               5211
                                                                                         ... 
Campus 2017 Staff: Teacher Regular Program Full Time Equiv Count                          230
Campus 2017 Staff: Assistant Principal Total Full Time Equiv - Experience Calculation     230
Campus 2017 Staff: Principal Total Full Time Equiv - Experience Calculation               230
Campus 2017 Staff: Teacher Total Full Time Equiv - Experience Calculation                 230
Campus Number                                               

In [73]:
# Get a list of columns with more than 230 NaNs
list[campus_NaCounts[campus_NaCounts.sort_values(ascending=False) > 230].index.values]

list[array(['Campus 2017 Staff: Support Total Base Salary Average',
       'Campus 2017 Staff: School Admin Total Base Salary Average',
       'Campus 2017 Staff: Teacher Total Base Salary Average',
       'Campus 2017 Staff: Teacher Beginning Base Salary Average',
       'Campus 2017 Staff: Teacher 1-5 Years Base Salary Average',
       'Campus 2017 Staff: Teacher 6-10 Years Base Salary Average',
       'Campus 2017 Staff: Teacher 11-20 Years Base Salary Average',
       'Campus 2017 Staff: Teacher > 20 Years Base Salary Average',
       'Campus 2017 Staff: Teacher Tenure Average',
       'Campus 2017 Staff: Teacher Experience Average',
       'Campus 2017 Staff: Principal Tenure Average',
       'Campus 2017 Staff: Principal Experience Average',
       'Campus 2017 Staff: Assistant Principal Tenure Average',
       'Campus 2017 Staff: Assistant Principal Experience Average',
       'Campus 2017 Class Size: Sec English - Avg Size',
       'Campus 2017 Class Size: Sec Math - Avg Size',

In [74]:
# Drop columns with a lot of NaNs
campus.drop(['Campus 2017 Staff: Support Total Base Salary Average',
       'Campus 2017 Staff: School Admin Total Base Salary Average',
       'Campus 2017 Staff: Teacher Total Base Salary Average',
       'Campus 2017 Staff: Teacher Beginning Base Salary Average',
       'Campus 2017 Staff: Teacher 1-5 Years Base Salary Average',
       'Campus 2017 Staff: Teacher 6-10 Years Base Salary Average',
       'Campus 2017 Staff: Teacher 11-20 Years Base Salary Average',
       'Campus 2017 Staff: Teacher > 20 Years Base Salary Average',
       'Campus 2017 Staff: Teacher Tenure Average',
       'Campus 2017 Staff: Teacher Experience Average',
       'Campus 2017 Staff: Principal Tenure Average',
       'Campus 2017 Staff: Principal Experience Average',
       'Campus 2017 Staff: Assistant Principal Tenure Average',
       'Campus 2017 Staff: Assistant Principal Experience Average',
       'Campus 2017 Class Size: Sec English - Avg Size',
       'Campus 2017 Class Size: Sec Math - Avg Size',
       'Campus 2017 Class Size: Sec Sci - Avg Size',
       'Campus 2017 Class Size: Sec Soc Stud- Avg Size',
       'Campus 2017 Class Size: Sec For Lng - Avg Size',
       'Campus 2017 Class Size: Kindergarten- Avg Size',
       'Campus 2017 Class Size: Grade 1 - Avg Size',
       'Campus 2017 Class Size: Grade 2 - Avg Size',
       'Campus 2017 Class Size: Grade 3 - Avg Size',
       'Campus 2017 Class Size: Grade 4 - Avg Size',
       'Campus 2017 Class Size: Grade 5 - Avg Size',
       'Campus 2017 Class Size: Grade 6 - Avg Size',
       'Campus 2017 Class Size: Mixed Elem. - Avg Size'], axis=1, inplace=True)

In [75]:
# Get Columns that need to be Standardized 
stand_cols = campus.drop(['Campus Number'], axis=1).columns

In [76]:
# Standarized columns 
campus[stand_cols]

standarized = StandardScaler()
standarized_fit = standarized.fit_transform(campus[stand_cols])

In [77]:
df_standarized = pd.DataFrame(data=standarized_fit, columns=stand_cols)
df_standarized_suf = df_standarized.add_suffix('_stand')

In [78]:
campus_stand = pd.concat([campus, df_standarized_suf], axis=1)
campus_stand.drop(columns=stand_cols, inplace=True)

In [79]:
df_campus = df.merge(campus_stand, left_on='2017 Campus ID', right_on='Campus Number')

In [80]:
# Drop column that identifice the school directly
df_campus.drop('Campus Number', axis=1, inplace=True)

In [81]:
# Drop NaNs from dataset so that dataframe could be run though models
df_campus.isna().sum().sum()

df_campus.dropna(inplace=True)

In [82]:
# Keep only EcoDis Students
df_campus = df_campus[df_campus['2017 EcoDis_EcoDis'] == 1]

In [83]:
# Since all students are EcoDis=1 we do not need thiese columns 
df_campus.drop(['2017 EcoDis_EcoDis','2017 EcoDis_Non-EcoDis'], axis=True, inplace=True)

In [84]:
df_campus.head()

,2017 Campus ID,Student ID,2019 Reading Meets,Consistent School Type,2017 Sex_F,2017 Sex_M,2017 Ethnicity_Black,2017 Ethnicity_Hispanic,2017 Ethnicity_Other Race,2017 Ethnicity_White,...,Campus 2017 Staff: Teacher White Full Time Equiv Percent_stand,Campus 2017 Staff: Teacher African American Full Time Equiv Percent_stand,Campus 2017 Staff: Teacher Hispanic Full Time Equiv Percent_stand,Campus 2017 Staff: Teacher Pacific Islander Full Time Equiv Percent_stand,Campus 2017 Staff: Teacher American Indian Full Time Equiv Percent_stand,Campus 2017 Staff: Teacher Asian Full Time Equiv Percent_stand,Campus 2017 Staff: Teacher Two or more races Full Time Equiv Percent_stand,Campus 2017 Staff: Teacher Male Full Time Equiv Percent_stand,Campus 2017 Staff: Teacher Female Full Time Equiv Percent_stand,Campus 2017 Staff: Teacher Student Ratio_stand
0,'057914113,QAHH18V4H,1.0,Public,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.048131,-0.462151,0.310124,-0.12128,1.336216,0.166834,-0.434903,-0.55814,0.559362,0.825588
1,'057914113,H04FQ8V40,1.0,Public,0.0,1.0,1.0,0.0,0.0,0.0,...,-0.048131,-0.462151,0.310124,-0.12128,1.336216,0.166834,-0.434903,-0.55814,0.559362,0.825588
2,'057914113,2A4H1854$,1.0,Public,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.048131,-0.462151,0.310124,-0.12128,1.336216,0.166834,-0.434903,-0.55814,0.559362,0.825588
3,'057914113,ACQ328242,1.0,Public,0.0,1.0,1.0,0.0,0.0,0.0,...,-0.048131,-0.462151,0.310124,-0.12128,1.336216,0.166834,-0.434903,-0.55814,0.559362,0.825588
4,'057914113,102FQ8V40,1.0,Public,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.048131,-0.462151,0.310124,-0.12128,1.336216,0.166834,-0.434903,-0.55814,0.559362,0.825588


## Public and Charter School Data

In [85]:
df_Public = df_campus[df_campus['Consistent School Type'] == 'Public']

In [86]:
df_Charter = df_campus[df_campus['Consistent School Type'] == 'Charter']

In [87]:
df_Public = df_Public.head(100)

In [88]:
df_Charter = df_Charter.head(100)

In [89]:
df_Public

,2017 Campus ID,Student ID,2019 Reading Meets,Consistent School Type,2017 Sex_F,2017 Sex_M,2017 Ethnicity_Black,2017 Ethnicity_Hispanic,2017 Ethnicity_Other Race,2017 Ethnicity_White,...,Campus 2017 Staff: Teacher White Full Time Equiv Percent_stand,Campus 2017 Staff: Teacher African American Full Time Equiv Percent_stand,Campus 2017 Staff: Teacher Hispanic Full Time Equiv Percent_stand,Campus 2017 Staff: Teacher Pacific Islander Full Time Equiv Percent_stand,Campus 2017 Staff: Teacher American Indian Full Time Equiv Percent_stand,Campus 2017 Staff: Teacher Asian Full Time Equiv Percent_stand,Campus 2017 Staff: Teacher Two or more races Full Time Equiv Percent_stand,Campus 2017 Staff: Teacher Male Full Time Equiv Percent_stand,Campus 2017 Staff: Teacher Female Full Time Equiv Percent_stand,Campus 2017 Staff: Teacher Student Ratio_stand
0,'057914113,QAHH18V4H,1.0,Public,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.048131,-0.462151,0.310124,-0.12128,1.336216,0.166834,-0.434903,-0.558140,0.559362,0.825588
1,'057914113,H04FQ8V40,1.0,Public,0.0,1.0,1.0,0.0,0.0,0.0,...,-0.048131,-0.462151,0.310124,-0.12128,1.336216,0.166834,-0.434903,-0.558140,0.559362,0.825588
2,'057914113,2A4H1854$,1.0,Public,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.048131,-0.462151,0.310124,-0.12128,1.336216,0.166834,-0.434903,-0.558140,0.559362,0.825588
3,'057914113,ACQ328242,1.0,Public,0.0,1.0,1.0,0.0,0.0,0.0,...,-0.048131,-0.462151,0.310124,-0.12128,1.336216,0.166834,-0.434903,-0.558140,0.559362,0.825588
4,'057914113,102FQ8V40,1.0,Public,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.048131,-0.462151,0.310124,-0.12128,1.336216,0.166834,-0.434903,-0.558140,0.559362,0.825588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,'057914116,1A1$18341,0.0,Public,0.0,1.0,1.0,0.0,0.0,0.0,...,-0.465526,0.291091,0.379442,-0.12128,1.336216,-0.407303,-0.434903,-0.535046,0.536845,0.357598
203,'057914116,1HHH1824Q,0.0,Public,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.465526,0.291091,0.379442,-0.12128,1.336216,-0.407303,-0.434903,-0.535046,0.536845,0.357598
204,'057914116,5Q1018H4$,0.0,Public,1.0,0.0,0.0,1.0,0.0,0.0,...,-0.465526,0.291091,0.379442,-0.12128,1.336216,-0.407303,-0.434903,-0.535046,0.536845,0.357598
205,'057914116,0V5H18V40,0.0,Public,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.465526,0.291091,0.379442,-0.12128,1.336216,-0.407303,-0.434903,-0.535046,0.536845,0.357598


## Features as Predictors for Public School Data

In [90]:
X_Public = df_Public[df_Public.columns.difference(['2017 Campus ID', 'Consistent School Type', '2019 Reading Meets'])]
X_Charter = df_Charter[df_Charter.columns.difference(['2017 Campus ID', 'Consistent School Type', '2019 Reading Meets'])]
X_Public.shape

(100, 125)

In [91]:
# List of Features as Predictors
list(X_Public.columns)

['2017 Ethnicity_Black',
 '2017 Ethnicity_Hispanic',
 '2017 Ethnicity_Other Race',
 '2017 Ethnicity_White',
 '2017 Reading Overall Score_Approaches',
 '2017 Reading Overall Score_Did Not Meet',
 '2017 Reading Overall Score_Masters',
 '2017 Reading Overall Score_Meets',
 '2017 Sex_F',
 '2017 Sex_M',
 'Campus 2017 Class Size: Grade 1 - # Classes_stand',
 'Campus 2017 Class Size: Grade 1 - # Students_stand',
 'Campus 2017 Class Size: Grade 2 - # Classes_stand',
 'Campus 2017 Class Size: Grade 2 - # Students_stand',
 'Campus 2017 Class Size: Grade 3 - # Classes_stand',
 'Campus 2017 Class Size: Grade 3 - # Students_stand',
 'Campus 2017 Class Size: Grade 4 - # Classes_stand',
 'Campus 2017 Class Size: Grade 4 - # Students_stand',
 'Campus 2017 Class Size: Grade 5 - # Classes_stand',
 'Campus 2017 Class Size: Grade 5 - # Students_stand',
 'Campus 2017 Class Size: Grade 6 - # Classes_stand',
 'Campus 2017 Class Size: Grade 6 - # Students_stand',
 'Campus 2017 Class Size: Kindergarten- # Clas

In [92]:
# Predicted Variable
y_Public = df_Public['2019 Reading Meets']
y_Charter = df_Charter['2019 Reading Meets']
y_Public

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
202    0.0
203    0.0
204    0.0
205    0.0
206    0.0
Name: 2019 Reading Meets, Length: 100, dtype: float64

In [93]:
# Public Data Features and Unique ID (Student ID)
X_Public.to_excel('Data/X_Public100_Clean.xlsx')
y_Public.to_excel('Data/y_Public100_Clean.xlsx')

In [94]:
# Charter Data Features and Unique ID (Student ID)
X_Charter.to_excel('Data/X_CharterPublic100_Clean.xlsx')
y_Charter.to_excel('Data/y_Charter100_Clean.xlsx')